In [58]:
# Imports using Sklearn make shortcut functions
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from datetime import datetime, timedelta

In [59]:
# Print current time
now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M:%S")

In [60]:
# Imports data
data = pd.read_csv('../processed_data/1988_to_2024_combined_clean_polling_and_results.csv')

In [61]:
# Convert date columns to datetime
data['enddate'] = pd.to_datetime(data['enddate'])
data['next_elec_date'] = pd.to_datetime(data['next_elec_date'])
data['startdate'] = pd.to_datetime(data['startdate'])

In [62]:
# Handle numerical transformer
num_columns_selector = ['samplesize', 'months_to_elec_weight']
num_transformer = MinMaxScaler()

In [63]:
# Handle categorical transformer
cat_columns_selector = ['rating']
cat_transformer = make_pipeline(OrdinalEncoder(categories = [['F','F+','E-','E','E+','D-','D','D+','C-','C','C+','B-','B','B+','A-']]),MinMaxScaler())

In [64]:
# One-hot encoder for party_in_power
ohe = OneHotEncoder()

In [65]:
# Build the preprocessing pipeline
preproc_pipeline = make_column_transformer(
    (num_transformer, num_columns_selector),
    (cat_transformer, cat_columns_selector),
    (ohe,['party_in_power']),
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [66]:
data['next_elec_date'].value_counts()

next_elec_date
2015-05-07    1930
2024-07-04    1397
2010-05-06     584
1992-04-09     453
2019-12-12     440
1997-05-01     338
2017-06-08     254
2001-06-07     213
2005-05-05      98
Name: count, dtype: int64

In [67]:
# Define election dates
election_date= datetime.strptime('2010-05-06', '%Y-%m-%d')
cutoff_date = election_date - timedelta(days=54)
prediction_date = election_date - timedelta(days=24)

In [68]:
# Split data
data_train = data[data['startdate'] > '2003-12-31']
data_train = data_train[data_train['startdate'] < cutoff_date]
data_test_1 = data[(data['startdate'] >= cutoff_date) & (data['startdate'] < prediction_date)]
data_test = data_test_1[data_test_1['next_elec_date'] == election_date]

In [69]:
# Fit transform preprocessing pipeline to data_train
data_train_processed = preproc_pipeline.fit_transform(data_train)


In [70]:
# Transform preprocessing pipeline to data_test
data_test_processed = preproc_pipeline.transform(data_test)

In [71]:
# Check feature names make sense for data_train
data_train_processed = pd.DataFrame(
    data_train_processed, columns=preproc_pipeline.get_feature_names_out()
)

# Check feature names make sense for data_test
data_test_processed = pd.DataFrame(
    data_test_processed, columns=preproc_pipeline.get_feature_names_out()
)

In [72]:
# Define our X by dropping irrelevant and y columns
X_train = data_train_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])
X_test = data_test_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])

In [73]:
# Build our target matrix
y_train = data_train_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]
y_test = data_test_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]

In [74]:
# Drops y_train actuals where the actual is the actual result we are trying to predict, replaces with NaNs
y_train.loc[y_train['next_elec_date'] == '2010-05-06',
         ['LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']] = np.nan

In [75]:
X_test.tail(10)

,samplesize,months_to_elec_weight,rating,poll_length,CON_FC,LAB_FC,LIB_FC,BRX_FC,GRE_FC,OTH_FC,PLC_FC,SNP_FC,UKI_FC
41,0.200847,0.831387,1.0,1,0.4,0.32,0.18,NaN,NaN,0.1,NaN,NaN,NaN
42,0.11012,0.831387,0.5,2,0.37,0.31,0.2,NaN,NaN,0.11,NaN,NaN,NaN
43,0.105662,0.831387,0.928571,1,0.37,0.3,0.2,NaN,NaN,0.13,NaN,NaN,NaN
44,0.206197,0.831387,1.0,1,0.37,0.31,0.2,NaN,NaN,0.12,NaN,NaN,NaN
45,0.329024,0.831387,0.0,1,0.38,0.28,0.22,NaN,NaN,0.12,NaN,NaN,NaN
46,0.288676,0.831387,1.0,3,0.39,0.31,0.17,NaN,NaN,0.13,NaN,NaN,NaN
47,0.105216,0.831387,0.928571,1,0.36,0.31,0.19,NaN,NaN,0.14,NaN,NaN,NaN
48,0.214668,0.831387,1.0,1,0.39,0.33,0.2,NaN,NaN,0.08,NaN,NaN,NaN
49,0.221355,0.831387,0.571429,5,0.36,0.27,0.23,NaN,NaN,0.14,NaN,NaN,NaN
50,0.308961,0.831387,0.428571,6,0.36,0.33,0.22,NaN,NaN,0.09,NaN,NaN,NaN


In [76]:
# Calculates average median value of X_test
averages = X_test.mean()
averages

samplesize               0.223375
months_to_elec_weight    0.783038
rating                   0.763305
poll_length              1.803922
CON_FC                   0.376863
LAB_FC                   0.305294
LIB_FC                    0.19451
BRX_FC                        NaN
GRE_FC                        NaN
OTH_FC                   0.123137
PLC_FC                        NaN
SNP_FC                        NaN
UKI_FC                        NaN
dtype: object

In [77]:
# Creates imputation values for y_train to impute over actuals we are trying to predict
imputation_values = {
    'CON_ACT': averages['CON_FC'],
    'LAB_ACT': averages['LAB_FC'],
    'LIB_ACT': averages['LIB_FC'],
    'BRX_ACT': 0, # doesnt exist in 2015
    'GRE_ACT': 0.006,
    'OTH_ACT': averages['OTH_FC'],
    'PLC_ACT':    0.006,
    'SNP_ACT': 0.04, # imputes value due to low forecasts
    'UKI_ACT': 0
}

In [78]:
# Applies imputation values to y_train
y_train = y_train.fillna(value=imputation_values)


/tmp/ipykernel_281396/3014033841.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = y_train.fillna(value=imputation_values)


In [79]:
# Instantiate the model for each party
models = {}
parties = ['CON', 'LAB', 'LIB', 'BRX', 'GRE', 'SNP', 'UKI', 'PLC', 'OTH']
for party in parties:
    models[party] = XGBRegressor(
        learning_rate=0.3, n_estimators=300, max_depth=3, subsample=0.7,
        objective='reg:squarederror', nthread=-1, enable_categorical=True
    )

In [80]:
# Train models
X_train_matrix = np.array(X_train)
for party in parties:
    models[party].fit(X_train_matrix, y_train[f'{party}_ACT'])

In [81]:
actuals_2010 = {
    "CON": 0.362,  # Conservative Party
    "LAB": 0.292,  # Labour Party
    "LIB": 0.235,  # Liberal Democrats
    "BRX": 0.000,  # Brexit Party (did not exist in 2010)
    "GRE": 0.009,  # Green Party
    "SNP": 0.019,  # Scottish National Party
    "UKI": 0.030,  # UK Independence Party
    "PLC": 0.005,  # Plaid Cymru
    "OTH": 0.048   # Other parties
}

In [82]:
# Evaluate predictions
X_test_matrix = np.array(X_test)
mean_predictions = {}
for party in parties:
    mean_predictions[party] = models[party].predict(X_test_matrix).mean()


In [83]:
# Print mean predictions
for party in parties:
    print(f"2010 mean prediction for {party}: {mean_predictions[party]}")
    print(f"2010 actual result for {party}  : {actuals_2010[party]}")


2010 mean prediction for CON: 0.37036293745040894
2010 actual result for CON  : 0.362
2010 mean prediction for LAB: 0.31037789583206177
2010 actual result for LAB  : 0.292
2010 mean prediction for LIB: 0.1976335197687149
2010 actual result for LIB  : 0.235
2010 mean prediction for BRX: 0.0
2010 actual result for BRX  : 0.0
2010 mean prediction for GRE: 0.006581234745681286
2010 actual result for GRE  : 0.009
2010 mean prediction for SNP: 0.03674643114209175
2010 actual result for SNP  : 0.019
2010 mean prediction for UKI: 0.0028239248786121607
2010 actual result for UKI  : 0.03
2010 mean prediction for PLC: 0.00606277072802186
2010 actual result for PLC  : 0.005
2010 mean prediction for OTH: 0.11494419723749161
2010 actual result for OTH  : 0.048


In [56]:
# Evaluate last five polls
last_predictions = {}
for party in parties:
    last_predictions[party] = models[party].predict(X_test_matrix)[-5:]

In [57]:
# Print mean of predictions
for party in parties:
    print(f"2010 last prediction for {party}: {last_predictions[party].mean()}")
    print(f"2010 actual result for {party}  : {actuals_2010[party]}")


2010 last prediction for CON: 0.362625390291214
2010 actual result for CON  : 0.362
2010 last prediction for LAB: 0.3198012709617615
2010 actual result for LAB  : 0.292
2010 last prediction for LIB: 0.1978214830160141
2010 actual result for LIB  : 0.235
2010 last prediction for BRX: 0.0
2010 actual result for BRX  : 0.0
2010 last prediction for GRE: 0.007135903928428888
2010 actual result for GRE  : 0.009
2010 last prediction for SNP: 0.03291087970137596
2010 actual result for SNP  : 0.019
2010 last prediction for UKI: 0.006471552886068821
2010 actual result for UKI  : 0.03
2010 last prediction for PLC: 0.006103444378823042
2010 actual result for PLC  : 0.005
2010 last prediction for OTH: 0.10538119077682495
2010 actual result for OTH  : 0.048
